In [1]:
import featuretools as ft

In [2]:
data = ft.demo.load_mock_customer()

In [4]:
data

{'customers':    customer_id zip_code           join_date date_of_birth
 0            1    60091 2011-04-17 10:48:33    1994-07-18
 1            2    13244 2012-04-15 23:31:04    1986-08-18
 2            3    13244 2011-08-13 15:42:34    2003-11-21
 3            4    60091 2011-04-08 20:08:14    2006-08-15
 4            5    60091 2010-07-17 05:27:50    1984-07-28,
 'sessions':     session_id  customer_id   device       session_start
 0            1            2  desktop 2014-01-01 00:00:00
 1            2            5   mobile 2014-01-01 00:17:20
 2            3            4   mobile 2014-01-01 00:28:10
 3            4            1   mobile 2014-01-01 00:44:25
 4            5            4   mobile 2014-01-01 01:11:30
 5            6            1   tablet 2014-01-01 01:23:25
 6            7            3   tablet 2014-01-01 01:39:40
 7            8            4   tablet 2014-01-01 01:55:55
 8            9            1  desktop 2014-01-01 02:15:25
 9           10            2   tablet 20

In [5]:
data_customers = data["customers"]
data_customers.head()

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15
4,5,60091,2010-07-17 05:27:50,1984-07-28


In [7]:
data_sessions = data["sessions"]
data_sessions.head()

,session_id,customer_id,device,session_start
0,1,2,desktop,2014-01-01 00:00:00
1,2,5,mobile,2014-01-01 00:17:20
2,3,4,mobile,2014-01-01 00:28:10
3,4,1,mobile,2014-01-01 00:44:25
4,5,4,mobile,2014-01-01 01:11:30


In [8]:
data_transactions = data["transactions"]
data_transactions.head()

,transaction_id,session_id,transaction_time,product_id,amount
0,298,1,2014-01-01 00:00:00,5,127.64
1,2,1,2014-01-01 00:01:05,2,109.48
2,308,1,2014-01-01 00:02:10,3,95.06
3,116,1,2014-01-01 00:03:15,4,78.92
4,371,1,2014-01-01 00:04:20,3,31.54


In [9]:
# 用数据集中的所有实体指定一个字典
entities = {
    "customers":(data_customers,"customer_id"),
    "sessions":(data_sessions,"session_id","session_start"),
    "transactions":(data_transactions,"transaction_id","transaction_time")
}

In [11]:
# 指定实体的关联方式 (最少输入一组entity)
relationships = [
    ("sessions","session_id","transactions","session_id"),
    ("customers","customer_id","sessions","customer_id")
]

In [12]:
# 运行深度特征合成
# DFS的最小输入包括一组entity，一组关系以及要计算特征的target_entity。

feature_matrix_customers,features_defs = ft.dfs(entities = entities,
                                               relationships=relationships,
                                               target_entity="customers")

In [13]:
feature_matrix_customers

,zip_code,COUNT(sessions),NUM_UNIQUE(sessions.device),MODE(sessions.device),SUM(transactions.amount),STD(transactions.amount),MAX(transactions.amount),SKEW(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),...,NUM_UNIQUE(sessions.WEEKDAY(session_start)),MODE(sessions.MODE(transactions.product_id)),MODE(sessions.DAY(session_start)),MODE(sessions.YEAR(session_start)),MODE(sessions.MONTH(session_start)),MODE(sessions.WEEKDAY(session_start)),NUM_UNIQUE(transactions.sessions.customer_id),NUM_UNIQUE(transactions.sessions.device),MODE(transactions.sessions.customer_id),MODE(transactions.sessions.device)
customer_id,,,,,,,,,,,,,,,,,,,,,
1,60091,8,3,mobile,9025.62,40.442059,139.43,0.019698,5.81,71.631905,...,1,4,1,2014,1,2,1,3,1,mobile
2,13244,7,3,desktop,7200.28,37.705178,146.81,0.098259,8.73,77.422366,...,1,3,1,2014,1,2,1,3,2,desktop
3,13244,6,3,desktop,6236.62,43.683296,149.15,0.418230,5.89,67.060430,...,1,1,1,2014,1,2,1,3,3,desktop
4,60091,8,3,mobile,8727.68,45.068765,149.95,-0.036348,5.73,80.070459,...,1,1,1,2014,1,2,1,3,4,mobile
5,60091,6,3,mobile,6349.66,44.095630,149.02,-0.025941,7.55,80.375443,...,1,3,1,2014,1,2,1,3,5,mobile


In [16]:
# 修改target entity
# DFS如此强大的原因之一是它可以为数据中的任何实体创建一个特征矩阵。例如，我们同样可以为session构建特征

feature_matrix_sessions,features_defs = ft.dfs(entities=entities,
                                              relationships=relationships,
                                              target_entity="sessions")

In [18]:
feature_matrix_sessions.head()

,customer_id,device,SUM(transactions.amount),STD(transactions.amount),MAX(transactions.amount),SKEW(transactions.amount),MIN(transactions.amount),MEAN(transactions.amount),COUNT(transactions),NUM_UNIQUE(transactions.product_id),...,customers.NUM_UNIQUE(transactions.product_id),customers.MODE(transactions.product_id),customers.DAY(join_date),customers.DAY(date_of_birth),customers.YEAR(join_date),customers.YEAR(date_of_birth),customers.MONTH(join_date),customers.MONTH(date_of_birth),customers.WEEKDAY(join_date),customers.WEEKDAY(date_of_birth)
session_id,,,,,,,,,,,,,,,,,,,,,
1,2,desktop,1229.01,41.600976,141.66,0.295458,20.91,76.813125,16,5,...,5,4,15,18,2012,1986,4,8,6,0
2,5,mobile,746.96,45.893591,135.25,-0.160550,9.32,74.696000,10,5,...,5,5,17,28,2010,1984,7,7,5,5
3,4,mobile,1329.00,46.240016,147.73,-0.324012,8.70,88.600000,15,5,...,5,2,8,15,2011,2006,4,8,4,1
4,1,mobile,1613.93,40.187205,129.00,0.234349,6.29,64.557200,25,5,...,5,4,17,18,2011,1994,4,7,6,0
5,4,mobile,777.02,48.918663,139.20,0.336381,7.43,70.638182,11,5,...,5,2,8,15,2011,2006,4,8,4,1
